In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
from copy import deepcopy
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
trade_df = pd.read_csv('/Users/the_networks_of_war/data_sources/Dyadic_COW_4.0.csv', encoding = 'utf8')

trade_df.rename({'flow1': 'money_flow_1', 'flow2': 'money_flow_2',
         'ccode1': 'c_code_1', 'ccode2': 'c_code_2',
         'importer1': 'state_name_1', 'importer2': 'state_name_2'},
        axis = 1, inplace = True)

trade_df['c_code_1'] = trade_df['c_code_1'].astype(int)
trade_df['c_code_2'] = trade_df['c_code_2'].astype(int)

trade_df['money_flow_1'] = ([s * 1000000 for s in trade_df['money_flow_1']])
trade_df['money_flow_2'] = ([s * 1000000 for s in trade_df['money_flow_2']])

trade_df.loc[trade_df['money_flow_1'] == -9000000, 'money_flow_1'] = None
trade_df.loc[trade_df['money_flow_2'] == -9000000, 'money_flow_2'] = None

trade_df = trade_df.sort_values(by = 'year', ascending = True).reset_index()
trade_df.drop(['source1', 'source2', 'bel_lux_alt_flow1', 'bel_lux_alt_flow2', 'china_alt_flow1', 'china_alt_flow2', 'version', 'index'], axis = 1, inplace = True)

In [5]:
trade_df_copy = deepcopy(trade_df)

trade_df_copy.rename({'c_code_1': 'c_code_1_copy', 'c_code_2':'c_code_2_copy',
         'state_name_1': 'state_name_1_copy', 'state_name_2': 'state_name_2_copy',
         'money_flow_1': 'money_flow_1_copy', 'money_flow_2': 'money_flow_2_copy'},
        axis = 1, inplace = True)

trade_df_copy.rename({'c_code_1_copy': 'c_code_2', 'c_code_2_copy':'c_code_1',
         'state_name_1_copy': 'state_name_2', 'state_name_2_copy': 'state_name_1',
         'money_flow_1_copy': 'money_flow_2', 'money_flow_2_copy': 'money_flow_1'},
        axis = 1, inplace = True)

In [6]:
trade_df = pd.concat([trade_df, trade_df_copy], axis = 0)
trade_df = trade_df.sort_values(by = 'year', ascending = True).reset_index()
trade_df.drop(['index'], axis = 1, inplace = True)

In [7]:
len(trade_df)

1773656

In [8]:
trade_df.head()

,c_code_1,c_code_2,dip1,dip2,money_flow_1,money_flow_2,smoothflow1,smoothflow2,smoothtotrade,spike1,spike2,state_name_1,state_name_2,tradedip,trdspike,year
0,70,90,0,0,nan,nan,-9.00,-9.00,-9.00,0,0,Mexico,Guatemala,0,0,1870
1,225,155,0,0,nan,nan,-9.00,-9.00,-9.00,0,0,Switzerland,Chile,0,0,1870
2,220,155,0,0,nan,nan,-9.00,-9.00,-9.00,0,0,France,Chile,0,0,1870
3,211,155,0,0,nan,nan,-9.00,-9.00,-9.00,0,0,Belgium,Chile,0,0,1870
4,210,155,0,0,nan,nan,-9.00,-9.00,-9.00,0,0,Netherlands,Chile,0,0,1870


In [8]:
# trade_df.to_pickle('../Pickles/trade_df.pkl')

In [9]:
trade_combo_df = deepcopy(trade_df[trade_df['year'] >= 1946][['c_code_1', 'c_code_2']])

duplicate_list = ['c_code_1', 'c_code_2']

trade_combo_df.drop_duplicates(subset = duplicate_list, keep = 'first', inplace = True)
trade_combo_df = trade_combo_df.reset_index()
trade_combo_df.drop(['index'], axis = 1, inplace = True)

In [10]:
len(trade_combo_df)

40030

In [11]:
for i, code in enumerate(trade_combo_df['c_code_1']):
    trade_combo_df.loc[i, 'trade_combo'] = str(trade_combo_df['c_code_1'][i]) + ' ' + str(trade_combo_df['c_code_2'][i])

In [12]:
trade_combo_df['trade_combo'].unique()

array(['630 355', '640 130', '670 360', ..., '345 626', '80 626',
       '223 626'], dtype=object)

In [13]:
column_list = ['money_flow_1', 'money_flow_2']
quad_list = ['money_flow_1', 'money_flow_2']
trade_net_df = deepcopy(trade_df[trade_df['year'] >= 1946])

for c_code in trade_combo_df['trade_combo'].unique():
    for column in column_list:
        data_list = list(trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].values)
        if str(data_list).count('nan') == len(data_list):
            trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = 0
        elif column not in quad_list and str(data_list).count('nan') <= len(data_list) - 2:
            trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].interpolate()
        elif column in quad_list and str(data_list).count('nan') <= len(data_list) - 2:
            try:
                trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].interpolate(method = 'quadratic')
            except:
                trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].bfill()
                trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].ffill()
        data_list = list(trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].values)
        if str(data_list).count('nan') <= len(data_list) - 1:
            trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].bfill()            
            trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].ffill()            

In [14]:
trade_net_df.head()

,c_code_1,c_code_2,dip1,dip2,money_flow_1,money_flow_2,smoothflow1,smoothflow2,smoothtotrade,spike1,spike2,state_name_1,state_name_2,tradedip,trdspike,year
177342,630,355,0,0,0.00,0.00,-9.00,-9.00,-9.00,0,0,Iran,Bulgaria,0,0,1946
177343,640,130,0,0,0.00,0.00,-9.00,-9.00,-9.00,0,0,Turkey,Ecuador,0,0,1946
177344,670,360,0,0,0.00,0.00,-9.00,-9.00,-9.00,0,0,Saudi Arabia,Romania,0,0,1946
177345,920,211,0,0,3388292.10,8085695.30,-9.00,-9.00,-9.00,0,0,New Zealand,Belgium,0,0,1946
177346,345,290,0,0,23000000.00,28709999.00,-9.00,-9.00,-9.00,0,0,Yugoslavia,Poland,0,0,1946


In [15]:
# trade_net_df.to_pickle('../Pickles/trade_net_df.pkl')

In [16]:
# trade_net_df = pd.read_pickle('../Pickles/trade_net_df.pkl')

In [17]:
trade_net_df.loc[trade_net_df['money_flow_1'] <= 0, 'money_flow_1'] = None
trade_net_df.loc[trade_net_df['money_flow_2'] <= 0, 'money_flow_2'] = None

In [18]:
for c_code in trade_combo_df['trade_combo'].unique():
    for column in column_list:
        data_list = list(trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].values)
        if str(data_list).count('nan') <= len(data_list) - 1:
            trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].bfill()            
            trade_net_df.loc[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1])), column] = trade_net_df[(trade_net_df['c_code_1'] == int(c_code.split()[0])) & (trade_net_df['c_code_2'] == int(c_code.split()[1]))][column].ffill()            

In [19]:
trade_net_df[(trade_net_df['money_flow_1'] <= 0) | (trade_net_df['money_flow_2'] <= 0)]

,c_code_1,c_code_2,dip1,dip2,money_flow_1,money_flow_2,smoothflow1,smoothflow2,smoothtotrade,spike1,spike2,state_name_1,state_name_2,tradedip,trdspike,year


In [20]:
country_loc_df = pd.read_pickle('Pickles/country_loc_df.pkl')

trade_net_df = deepcopy(pd.merge(country_loc_df, trade_net_df, how = 'inner', on = ['c_code_1']))
trade_net_df.rename({'un_region': 'un_region_1', 'un_continent': 'un_continent_1'}, axis = 1, inplace = True)
trade_net_df.drop(['num_years'], axis = 1, inplace = True)

country_loc_df.rename({'c_code_1': 'c_code_2'}, axis = 1, inplace = True)

trade_net_df = deepcopy(pd.merge(country_loc_df, trade_net_df, how = 'inner', on = ['c_code_2']))
trade_net_df.rename({'un_region': 'un_region_2', 'un_continent': 'un_continent_2'}, axis = 1, inplace = True)
trade_net_df.drop(['num_years'], axis = 1, inplace = True)

FileNotFoundError: [Errno 2] No such file or directory: 'Pickles/country_loc_df.pkl'

In [ ]:
trade_net_df.to_pickle('../Pickles/trade_net_df.pkl')

In [ ]:
aggregations = {
    'c_code_2': 'count',
    'money_flow_1': 'sum',
    'money_flow_2': 'sum',
  }

trade_df_group = trade_df.groupby(['c_code_1', 'state_name_1', 'year']).agg(aggregations).reset_index()
trade_df_group.rename({'c_code_2': 'num_trade_states', 'money_flow_1': 'export_dollars', 'money_flow_2': 'import_dollars'}, axis = 1, inplace = True)

In [ ]:
trade_df_group.to_pickle('../Pickles/trade_df_group.pkl')

In [ ]:
print(len(trade_net_df[trade_net_df['money_flow_1'] <= 0]))
print(len(trade_net_df[trade_net_df['money_flow_2'] <= 0]))